In [1]:
from processing import preprocessing,get_movies_aspect_matrix,dict_movie_aspect,viewed_matrix
import json
import pandas as pd
import numpy as np
import scipy
import sklearn.preprocessing as pp
from compute_strength import film_strength
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from math import sqrt

In [2]:
import json
import pandas as pd
trainset = pd.read_csv('/Users/user/Downloads/thesis/book/train.csv')
testset = pd.read_csv('/Users/user/Downloads/thesis/book/test.csv')
#trainset['user_rating'] = (trainset['user_rating'] + 1) * 2 + 1
#testset['user_rating'] = (testset['user_rating'] + 1) * 2 + 1
def traite_train_test(df):
    df['authors'] = df['authors'].apply(lambda x: json.loads(x))
    df['genres'] = df['genres'].apply(lambda x: json.loads(x))
    return df
trainset = traite_train_test(trainset)
testset = traite_train_test(testset)
hehe_test = trainset.copy()
df_empty = testset.copy()
df_empty['user'] = df_empty['user'].astype('int')
df_empty['rating'] = df_empty['rating'].astype('float')
df_empty['item'] = df_empty['item'].astype('int')
hehe_test.index = range(len(hehe_test))
df_empty.index = range(len(df_empty))
hehe_test = hehe_test.rename(columns={'user':'userID','item':'itemID','rating':'cnt'})
df_empty = df_empty.rename(columns={'user':'userID','item':'itemID','rating':'cnt'})

In [3]:
df_all = pd.concat([hehe_test,df_empty])

In [4]:
df_sum = df_all.copy()
movie_set = set(df_sum['itemID'])
movie = df_sum[['itemID', 'authors', 'genres','cnt']].loc[df_sum[['itemID', 'authors', 'genres','cnt']].astype(str).drop_duplicates().index]
movie.index = range(len(movie))
movie = movie[['itemID', 'authors', 'genres','cnt']].loc[movie[['itemID', 'authors', 'genres','cnt']].astype(str).drop_duplicates().index]

In [5]:
ratings_dict = dict()
train_ratings_dict = dict()

train_ratings_dict["userID"] = []
train_ratings_dict["itemID"] = []
train_ratings_dict["rating"] = []


test_eva_dict = hehe_test.to_dict('records')
for row in test_eva_dict[:]:
    tuple_key = (row['userID'],row['itemID'])
    ratings_dict[tuple_key] = row['cnt']
    train_ratings_dict["userID"].append(row['userID'])
    train_ratings_dict["itemID"].append(row['itemID'])
    train_ratings_dict["rating"].append(row['cnt'])

In [6]:
compressed_test_ratings_dict = dict()
test_eva_dict = df_empty.to_dict('records')
for row in test_eva_dict[:]:
    if row['userID'] not in compressed_test_ratings_dict:
        compressed_test_ratings_dict[row['userID']] = []
    compressed_test_ratings_dict[row['userID']].append((row['itemID'],row['cnt']))

In [7]:
df_all.columns


Index(['userID', 'itemID', 'authors', 'genres', 'cnt'], dtype='object')

In [8]:
films = dict()


test_eva_dict = movie.to_dict('records')
for row in test_eva_dict[:]:
    if row['itemID'] not in films:
        films[row['itemID']] = dict()
    films[row['itemID']]['authors'] = row['authors']
    films[row['itemID']]['genres'] = row['genres']
    #films[row['itemID']]['rating'] = [row['cnt']]
    
    
    

In [9]:
movies_watched = viewed_matrix(train_ratings_dict, films, 'netflix')
movies_watched = pd.DataFrame.from_dict(
    movies_watched, dtype='int64', orient='index').T
movies_watched = movies_watched.replace(np.nan, 0)

In [10]:
user_ids_in_matrix = movies_watched.index.values
movies_watched = movies_watched.T
movies_watched = scipy.sparse.csc_matrix(movies_watched.values)
normalized_matrix_by_column = pp.normalize(
    movies_watched.tocsc(), norm='l2', axis=0)
cosine_sims = normalized_matrix_by_column.T * normalized_matrix_by_column
assert cosine_sims.shape[0] == cosine_sims.shape[1] == len(
    user_ids_in_matrix)

In [11]:
len(user_ids_in_matrix)

5575

In [12]:
sims = dict()
for i in user_ids_in_matrix:
    sims[i] = []
cosine_sims = cosine_sims.todok().items()

for ((row, col), sim) in cosine_sims:
    if row != col:
        sims[user_ids_in_matrix[row]].append(
                (user_ids_in_matrix[col], sim))

In [13]:
category_in_db, movies_all_authors_matrix = get_movies_aspect_matrix(
    films, "authors")
downloads_in_db, movies_all_genres_matrix = get_movies_aspect_matrix(
    films, "genres")


We have 1360 authors (an example is 932)
We have 305 genres (an example is 1608)


In [14]:
df_all.columns


Index(['userID', 'itemID', 'authors', 'genres', 'cnt'], dtype='object')

In [15]:
MUR = 0.5
MUC = 0.25
MUD = 0.25

In [16]:
predictions = []
for user_id, true_ratings in compressed_test_ratings_dict.items():
    if true_ratings:
        for (film_id, str_rating) in true_ratings:
            str_rating = str_rating 
            strength = film_strength(MUR, MUC, MUD, user_id, film_id, films, ratings_dict,
                                     sims[user_id],movies_all_genres_matrix,movies_all_authors_matrix)
            strength = strength 
            predictions.append((float(str_rating), float(strength)))



KeyError: 3630

In [ ]:
df_all[df_all['userID'] == 3630]

In [17]:
true_ratings = [x for (x, y) in predictions]
predicted_ratings = [round(y) for (x, y) in predictions]
sqrt(mean_squared_error(true_ratings, predicted_ratings)), mean_absolute_error(true_ratings, predicted_ratings)

(1.2825705514863124, 1.0417495029821073)